In [3]:
import pandas as pd
import pickle

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/abdala9512/.local/share/virtualenvs/mlops-zoomcamp-course-f09ZVlZ1/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/abdala9512/.local/share/virtualenvs/mlops-zoomcamp-course-f09ZVlZ1/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df = read_data('../data/yellow_tripdata_2022-02.parquet')

dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [5]:
import numpy as np

np.std(y_pred)

12.758556818790902

In [9]:
df['ride_id'] = f'{2022:04d}/{2:02d}_' + df.index.astype('str')
df["prediction"] = y_pred
df[["ride_id", "prediction"]]

,ride_id,prediction
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2979426,2022/02_2979426,12.038225
2979427,2022/02_2979427,11.441569
2979428,2022/02_2979428,11.890459
2979429,2022/02_2979429,15.102681


In [10]:
df_result = df[["ride_id", "prediction"]]
df_result.to_parquet(
    "output_predictions.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)